In [ ]:
# ai_players.py
from game_objects import Player
from utils import find_possible_groups
import random

class AIPlayer(Player):
    def choose_action(self, game_state):
        # Basic AI logic
        groups = find_possible_groups(self.hand)
        if groups:
            self.discard_group(groups[0], game_state['deck'])
        elif not self.actions_taken['draw']:
            num_to_draw = min(3, 20 - len(self.hand))
            self.draw_cards(game_state['deck'], num_to_draw)
            self.actions_taken['draw'] = True
        elif not self.actions_taken['steal']:
            other_players = [p for p in game_state['players'] if p != self and p.hand]
            if other_players:
                target_player = random.choice(other_players)
                self.take_card_from(target_player)
                self.actions_taken['steal'] = True
        else:
            self.reset_actions()
            # Move to next player handled in PlayScreen

class AggressiveAIPlayer(AIPlayer):
    def choose_action(self, game_state):
        if not self.actions_taken['steal']:
            other_players = [p for p in game_state['players'] if p != self and p.hand]
            if other_players:
                target_player = max(other_players, key=lambda p: len(p.hand))
                self.take_card_from(target_player)
                self.actions_taken['steal'] = True
                return
        super().choose_action(game_state)

class DefensiveAIPlayer(AIPlayer):
    def choose_action(self, game_state):
        groups = find_possible_groups(self.hand)
        if groups:
            self.discard_group(groups[0], game_state['deck'])
        elif not self.actions_taken['draw']:
            num_to_draw = min(3, 20 - len(self.hand))
            self.draw_cards(game_state['deck'], num_to_draw)
            self.actions_taken['draw'] = True
        else:
            super().choose_action(game_state)

class BalancedAIPlayer(AIPlayer):
    def choose_action(self, game_state):
        groups = find_possible_groups(self.hand)
        if groups:
            self.discard_group(groups[0], game_state['deck'])
        elif len(self.hand) < 15 and not self.actions_taken['draw']:
            num_to_draw = min(3, 20 - len(self.hand))
            self.draw_cards(game_state['deck'], num_to_draw)
            self.actions_taken['draw'] = True
        elif not self.actions_taken['steal']:
            other_players = [p for p in game_state['players'] if p != self and p.hand]
            if other_players:
                target_player = random.choice(other_players)
                self.take_card_from(target_player)
                self.actions_taken['steal'] = True
        else:
            super().choose_action(game_state)

Action 2

In [ ]:
# add this function to utils.py
import copy
def probability_of_valid_group_hand(players_hand, other_hand1, other_hand2 = None):
  # initialize variable
  success = 0
  prob = []

  #find prob. for 1st player
  if len(other_hand1) == 1:
    prob.append(0)
  else:
    for i in other_hand1:
      hands = copy.deepcopy(players_hand)
      hands.collection.append(i)
      if find_possible_groups(hands) != []:
        success += 1
    prob.append(success / len(other_hand1))

  #find prob. for 2nd player
  if other_hand2 is not None:
    if len(other_hand2) == 1:
      prob.append(0)
    else:
      success = 0
      for i in other_hand2:
        hands = copy.deepcopy(players_hand)
        hands.collection.append(i)
        if find_possible_groups(hands) != []:
          success += 1
      prob.append(success / len(other_hand2))

  return prob

Calling the function

In [ ]:
# directly copy and paste the code
class AggressiveAIPlayer(AIPlayer):
    def choose_action(self, game_state):
        if not self.actions_taken['steal']:
            other_players = [p for p in game_state['players']]
            target_player = None
            if other_players != []:
                if len(other_players) == 2:
                    prob = probability_of_valid_group_hand(self.hand, other_players[0].hand, other_players[1].hand)
                    if not (prob[0] == 0 and prob[1] == 0):
                      if prob[0] > prob[1]:
                        target_player = other_players[0]
                      elif prob[1] > prob[0]:
                        target_player = other_players[1]
                      else:
                        target_player = max(other_players, key=lambda p: len(p.hand))
                else:
                    prob = probability_of_valid_group_hand(self.hand, other_players[0].hand)
                    if prob[0] != 0:
                        target_player = other_players[0]
                if target_player is not None:
                    self.take_card_from(target_player)
                self.actions_taken['steal'] = True
        else:
          super().choose_action(game_state)

In [ ]:
# directly copy and paste the code
class BalancedAIPlayer(AIPlayer):
    def choose_action(self, game_state):
        groups = find_possible_groups(self.hand)
        if groups:
            self.discard_group(groups[0], game_state['deck'])
        elif len(self.hand) < 15 and not self.actions_taken['draw']:
            num_to_draw = min(3, 20 - len(self.hand))
            self.draw_cards(game_state['deck'], num_to_draw)
            self.actions_taken['draw'] = True
        elif not self.actions_taken['steal']:
            other_players = [p for p in game_state['players']]
            target_player = None
            if other_players != []:
                if len(other_players) == 2:
                    prob = probability_of_valid_group_hand(self.hand, other_players[0].hand, other_players[1].hand)
                    if not (prob[0] == 0 and prob[1] == 0):
                      if prob[0] > prob[1]:
                        target_player = other_players[0]
                      elif prob[1] > prob[0]:
                        target_player = other_players[1]
                      else:
                        target_player = max(other_players, key=lambda p: len(p.hand))
                else:
                    prob = probability_of_valid_group_hand(self.hand, other_players[0].hand)
                    if prob[0] != 0:
                        target_player = other_players[0]
                if target_player is not None:
                    self.take_card_from(target_player)
                self.actions_taken['steal'] = True
        else:
            super().choose_action(game_state)